In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv('./data/credits.csv')
df2 = pd.read_csv('./data/movies_metadata.csv')

C:\Users\m\AppData\Local\Temp\ipykernel_11160\248776730.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./data/movies_metadata.csv')


In [3]:
df2.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
df1.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [5]:
df2.id

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45466, dtype: object

In [6]:
ids = [id for id in df2.id if '-' in id]

ids

['1997-08-20', '2012-09-29', '2014-01-01']

In [7]:
for id in ids:
    df2 = df2[df2.id!=id]

In [8]:
df2['id'] = df2['id'].astype(int)

In [9]:
movie_credit_df=df2.merge(df1, on='id')

In [10]:
movie_credit_df.id

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45533    439050
45534    111109
45535     67758
45536    227506
45537    461257
Name: id, Length: 45538, dtype: int32

In [11]:
movie_credit_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

- This task is a content based recommendation system
- We will recommend movies to new users according to movie description/overview

In [12]:
movie_credit_df.overview

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45533          Rising and falling between a man and woman.
45534    An artist struggles to finish his work while a...
45535    When one of her hits goes wrong, a professiona...
45536    In a small town live two brothers, one a minis...
45537    50 years after decriminalisation of homosexual...
Name: overview, Length: 45538, dtype: object

------------------------------
* Preprocessing

1- Remove NAN values

In [13]:
movie_credit_df.overview = movie_credit_df.overview.fillna('')

In [42]:
ovs = [overview for overview in movie_credit_df.overview]
titles = [title for title in movie_credit_df.title]

print(titles[0])
print(ovs[0])

Toy Story
Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


2- Turn overviews into lowercase

In [44]:
ovs = [ov.lower() for ov in ovs]
titles = [str(title).lower() for title in titles]

In [45]:
print(titles[0])
print(ovs[0])

toy story
led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences.


3- Remove Punctuations

In [17]:
import re

ovs = [re.sub(r'[^\w\s]', '', ov) for ov in ovs]

ovs[0]

'led by woody andys toys live happily in his room until andys birthday brings buzz lightyear onto the scene afraid of losing his place in andys heart woody plots against buzz but when circumstances separate buzz and woody from their owner the duo eventually learns to put aside their differences'

4- Remove stopwords

In [18]:
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

* split overviews into tokens and search for stopwords

In [19]:
filtered_ovs = []
for ov in ovs:
    filtered_ov = []
    for t in ov.split():
        if t not in english_stopwords:
            filtered_ov.append(t)

    filtered_ovs.append(filtered_ov)
    
filtered_ovs

[['led',
  'woody',
  'andys',
  'toys',
  'live',
  'happily',
  'room',
  'andys',
  'birthday',
  'brings',
  'buzz',
  'lightyear',
  'onto',
  'scene',
  'afraid',
  'losing',
  'place',
  'andys',
  'heart',
  'woody',
  'plots',
  'buzz',
  'circumstances',
  'separate',
  'buzz',
  'woody',
  'owner',
  'duo',
  'eventually',
  'learns',
  'put',
  'aside',
  'differences'],
 ['siblings',
  'judy',
  'peter',
  'discover',
  'enchanted',
  'board',
  'game',
  'opens',
  'door',
  'magical',
  'world',
  'unwittingly',
  'invite',
  'alan',
  'adult',
  'whos',
  'trapped',
  'inside',
  'game',
  '26',
  'years',
  'living',
  'room',
  'alans',
  'hope',
  'freedom',
  'finish',
  'game',
  'proves',
  'risky',
  'three',
  'find',
  'running',
  'giant',
  'rhinoceroses',
  'evil',
  'monkeys',
  'terrifying',
  'creatures'],
 ['family',
  'wedding',
  'reignites',
  'ancient',
  'feud',
  'nextdoor',
  'neighbors',
  'fishing',
  'buddies',
  'john',
  'max',
  'meanwhile',

In [20]:
print(ovs[0])
print([' '.join(ov) for ov in filtered_ovs][0])

led by woody andys toys live happily in his room until andys birthday brings buzz lightyear onto the scene afraid of losing his place in andys heart woody plots against buzz but when circumstances separate buzz and woody from their owner the duo eventually learns to put aside their differences
led woody andys toys live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences


In [21]:
filtered_ovs = [' '.join(ov) for ov in filtered_ovs]

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

In [23]:
tfidf_matrix = tfidf.fit_transform(filtered_ovs[:20000])

In [24]:
tfidf_matrix.shape

(20000, 55270)

In [25]:
len(filtered_ovs[:20000])

20000

In [26]:
len(set(' '.join(filtered_ovs[:20000]).split()))

55492

NOTE: approiximitly the same dimensions of tfidf_matrix

* Calculate cosine similarity between every overview and another

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [29]:
cos_sim.shape

(20000, 20000)

In [30]:
indices = pd.Series(movie_credit_df.index, index=movie_credit_df['title']).drop_duplicates()

indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45533
Century of Birthing            45534
Betrayal                       45535
Satan Triumphant               45536
Queerama                       45537
Length: 45538, dtype: int64

NOTE: cos_sim matrix introduce similarity score between every movie and all movies in the dataset (20000 record)

In [76]:
def get_recommendation(title, n = 10):
    title = title.lower()

    ind = titles.index(title)

    all_sim_movies_scores = cos_sim[ind].tolist()

    all_sim_movies_scores.pop(all_sim_movies_scores.index(max(all_sim_movies_scores)))

    top_movies = []
    scores = []
    i = 0
    while i < n:
        score = max(all_sim_movies_scores)
        most_sim_ind = all_sim_movies_scores.index(score)
        top_movies.append(titles[most_sim_ind])
        scores.append(f'{round(score*100, 2)}%')
        all_sim_movies_scores.pop(most_sim_ind)
        i = i+1

    return dict(zip(top_movies, scores))

In [77]:
rec_dict = get_recommendation('Assassins')
pd.DataFrame(np.array([list(rec_dict.keys()), list(rec_dict.values())]).T, columns=['Movie', 'Similarity Score'])

,Movie,Similarity Score
0,and then there were none,30.88%
1,the last lions,16.47%
2,3 ring circus,12.64%
3,cemetery junction,11.69%
4,old men in new cars: in china they eat dogs ii,11.22%
5,american boy: a profile of: steven prince,11.08%
6,stricken,11.03%
7,fancy pants,10.85%
8,relative strangers,10.63%
9,wild west comedy show: 30 days & 30 nights - h...,10.43%


In [55]:
import string

for x in get_recommendation('Assassins'):
    print(movie_credit_df[movie_credit_df.original_title == string.capwords(x)]['genres'])

4856     [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...
37540    [{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...
Name: genres, dtype: object
18392    [{'id': 99, 'name': 'Documentary'}]
Name: genres, dtype: object
14263    [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object
15304    [{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...
Name: genres, dtype: object
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
8049    [{'id': 10402, 'name': 'Music'}, {'id': 35, 'n...
Name: genres, dtype: object
12149    [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
Series([], Name: genres, dtype: object)
18046    [{'id': 99, 'name': 'Documentary'}]
Name: genres, dtype: object
9335    [{'id': 35, 'name': 'Comedy'}]
